# **Analysis of Perturbations in Single-Cell RNA-Seq Data**

First, we create a `PertData` object to load and handle perturbation data.
We specify that we want to load the `norman` dataset.

In [ ]:
import pertdata as pt

norman = pt.PertData.from_repo(name="norman", save_dir="data")
print(norman)

# Analysis and Preparation of Single-Cell RNA-Seq Data

The actual perturbation data is stored in an [`AnnData`](https://anndata.readthedocs.io/en/latest/) object.
`AnnData` is specifically designed for matrix-like data.
By this we mean that we have $n$ observations, each of which can be represented as $d$-dimensional vectors, where each dimension corresponds to a variable or feature.
Both the rows and columns of this $n \times d$-matrix are special in the sense that they are indexed.
For instance, in scRNA-seq data, each row corresponds to a cell with a barcode, and each column corresponds to a gene with a gene identifier.

Here we show how to access the **gene expression matrix**, the **perturbations labels**, the **control labels**, and the **gene names**.

In [ ]:
print(norman)
print(norman.adata)

X = norman.adata.X
y_pert = norman.adata.obs["condition"]
y_ctrl = norman.adata.obs["control"]
gene_names = norman.adata.var["gene_name"]

print(f"X.shape={X.shape}")
print(f"y_pert.shape={y_pert.shape}")
print(f"y_ctrl.shape={y_ctrl.shape}")
print(f"gene_names.shape={gene_names.shape}")

In general, in a perturbation dataset, we find $k$ cell lines.
Usually, one cell line remains unperturbed, and the others are cultivated separately (with different perturbations, i.e., gene knockouts).
The mRNA of usually a few thousand cells of each cell line is sequenced (using a single-cell RNA sequencing protocol), generating the $n$ $d$-dimensional gene expression profiles.
In particular, perturbation labels are available (i.e., we know which genes were knocked out in each cell line).

Before we can take a closer look at our data, we need to fix the perturbation labels, because they might be expressed ambiguously (e.g., single-gene perturbations can be expressed as `ctrl+<gene1>` or `<gene1>+ctrl`, falsely leading to two distinct labels for the perturbation of `<gene1>`).

The `PertData` object already contains the fixed perturbation labels, which were computed during initialization.

In [ ]:
print(f"Unique perturbations (unfixed): {len(set(norman.adata.obs["condition"]))}")
print(f"Unique perturbations (fixed): {len(set(norman.adata.obs["condition_fixed"]))}")

Furthermore, we will work with single-gene perturbations only.
Hence, we have to filter out double-gene perturbations.
Double-gene perturbations can be identified by a `+` in the fixed perturbation labels.

In [4]:
filter_mask = ~norman.adata.obs["condition_fixed"].str.contains(r"\+")
indexes_to_keep = filter_mask[filter_mask].index
adata_single = norman.adata[indexes_to_keep].copy()

Let's take a closer look at the new data:

In [ ]:
print(adata_single)
print(f"Unique perturbations: {len(set(adata_single.obs['condition_fixed']))}")
print("Number of samples per condition:")
print(adata_single.obs["condition_fixed"].value_counts())

Because gene expression data is very sparse, i.e., the expression is often not measured successfully or correctly, we will limit our experiment to the 128 genes with the highest variances.

In [ ]:
# Number of top genes to select
d = 128

# Compute the gene variances
gene_variances = adata_single.X.toarray().var(axis=0)

# Sort the gene variances in descending order and get the indexes of the top d genes
sorted_indexes = gene_variances.argsort()[::-1]

# Get the indexes of the top d genes
top_gene_indexes = sorted_indexes[:d]

# Get the gene names of the top d genes
top_genes = adata_single.var["gene_name"].iloc[top_gene_indexes]

# Get the variances of the top d genes
top_variances = gene_variances[top_gene_indexes]

# Print the top d genes with the highest variances
print(f"Top {d} genes with highest variances:")
for gene, variance in zip(top_genes, top_variances):
    print(f"{gene:15}: {variance:.2f}")

# Create a new AnnData object with only the top d genes
adata_single_top_genes = adata_single[:, top_gene_indexes].copy()

# Training an Autoencoder on Single-Cell RNA-Seq Data

First, we prepare our data.

Note that we shuffle the data in the `train_loader` but not in the `test_loader`.

Shuffling the training data is a common practice to ensure that the model does not learn the order of the data.
It helps in breaking correlations by preventing the model from learning any unintended patterns or correlations that might exist in the order of the training data.

Shuffling is typically not used for the testing data because non-shuffled data ensures that the evaluation is consistent and reproducible.
The model is tested on the same data in the same order each time, which is important for comparing performance across different runs.

In [7]:
import torch
from torch.utils.data import DataLoader, TensorDataset, random_split

# Convert the gene expression matrix to a PyTorch tensor
X = torch.tensor(data=adata_single_top_genes.X.toarray(), dtype=torch.float32)

# Create a PyTorch dataset
dataset = TensorDataset(X, X)

# Create train and test datasets
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(
    dataset=dataset, lengths=[train_size, test_size]
)

# Number of workers
num_workers = 3  # Adjust based on the number of CPU cores

# Create train and test data loaders
train_loader = DataLoader(
    dataset=train_dataset,
    batch_size=64,
    shuffle=True,
    num_workers=num_workers,
    persistent_workers=True,
)
test_loader = DataLoader(
    dataset=test_dataset,
    batch_size=64,
    shuffle=False,
    num_workers=num_workers,
    persistent_workers=True,
)

In [ ]:
import pytorch_lightning as pl

from autoencoder import Autoencoder

# Get the number of features
n_features = X.shape[1]
print(f"n_features={n_features}")

# Get the number of samples
n_samples = X.shape[0]  # = len(train_dataset) + len(test_dataset) = len(dataset)
print(f"n_samples={n_samples}")

# Create the autoencoder
autoencoder = Autoencoder(in_features=n_features)

In [ ]:
from pytorch_lightning.loggers import CSVLogger

# Initialize the CSV logger
logger = CSVLogger(save_dir="lightning_logs", name="ae_experiment")

# Train the autoencoder
trainer = pl.Trainer(max_epochs=4, logger=logger)
trainer.fit(model=autoencoder, train_dataloaders=train_loader)

In [ ]:
# Test the autoencoder
trainer.test(model=autoencoder, dataloaders=test_loader)

In [ ]:
import matplotlib.pyplot as plt
import os
import pandas as pd


def plot_train_loss(logfile: str) -> None:
    print(f"logfile={logfile}")
    log = pd.read_csv(filepath_or_buffer=logfile)
    plt.plot(log["step"], log["train_loss"])
    plt.xlabel("Step")
    plt.ylabel("Train Loss")
    plt.show()


# Construct the path to the most recent version directory
most_recent_metrics_file = os.path.join(
    logger.save_dir, logger.name, f"version_{logger.version}", "metrics.csv"
)

# Plot the training loss
plot_train_loss(logfile=most_recent_metrics_file)